In [0]:
import uuid
from pyspark.sql.functions import pandas_udf, explode, udf,from_json
from pyspark.sql.types import ArrayType, StringType, DoubleType
import json
from pyspark.sql.functions import struct, flatten, lit, col,row_number, current_timestamp
from pyspark.sql import functions as F
from typing import Iterator, Tuple
import pandas as pd

from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType
from datetime import datetime
from pyspark.sql import Window


In [0]:

dbutils.widgets.text("catalog", "users")
dbutils.widgets.text("schema", "dom_rodrigues")

dbutils.widgets.dropdown("debug", "true", ["true", "false"])
dbutils.widgets.text("runId", "")
dbutils.widgets.text("s3_batch_copy_id", "Test1", "Batch Copy Id to compare")

catalog = dbutils.widgets.get("catalog")
schema =  dbutils.widgets.get("schema")
runId =  dbutils.widgets.get("runId")

runId = str(uuid.uuid4()) if len(runId.strip()) == 0 else runId

s3_batch_copy_id=dbutils.widgets.get("s3_batch_copy_id")

debug =  True if dbutils.widgets.get("debug") == 'true' else False
trace = True
createMetadataTables = False

spark.sql(f"use {catalog}.{schema}")

DataFrame[]

#### Test Data

In [0]:
if createMetadataTables:
  src_inventory_s3_location="s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY"
  src_bucket_name="databricks-e2demofieldengwest"
  src_prefix="dom_rodrigues/folder_1/"
  src_inventory_name="dom_rodrigues_inventory_test"

  dest_inventory_s3_location="s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY"
  dest_bucket_name="databricks-e2demofieldengwest"
  dest_prefix="dom_rodrigues/folder_2/"
  dest_inventory_name="dom_rodrigues_inventory_dest_test"

  data = [{
      "s3_batch_copy_id": s3_batch_copy_id,
      "src_inventory_s3_location": src_inventory_s3_location,
      "src_bucket_name": src_bucket_name,
      "src_prefix": src_prefix,
      "src_inventory_name": src_inventory_name,
      "dest_inventory_s3_location": dest_inventory_s3_location,
      "dest_bucket_name": dest_bucket_name,
      "dest_inventory_name": dest_inventory_name,
      "dest_prefix": dest_prefix,
      "last_recon_timestamp": datetime.now(),
      "runId": "0",
      "status": "",
      "recon_status" : "",
      "enabled": True
  }]

  # Create DataFrame from the list of dictionary
  df_s3_recon = spark.createDataFrame(data)
  df_s3_recon.display()

  df_s3_recon.write.option("mergeSchema", "true").mode("overwrite").format("delta").saveAsTable(f"{catalog}.{schema}.ucd_src_dest_mapping")

In [0]:
df_s3_recon = spark.read.table(f"{catalog}.{schema}.ucd_src_dest_mapping")
df_s3_recon = df_s3_recon.filter(f"s3_batch_copy_id = '{s3_batch_copy_id}' and enabled = 'true'")
df_s3_recon.display() if trace else None


dest_bucket_name,dest_inventory_name,dest_inventory_s3_location,dest_prefix,enabled,last_recon_timestamp,recon_status,runId,s3_batch_copy_id,src_bucket_name,src_inventory_name,src_inventory_s3_location,src_prefix,status
databricks-e2demofieldengwest,dom_rodrigues_inventory_dest_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_2/,true,2025-11-10T16:21:14.828907Z,"[""{""status"":""EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"",""count"":44}""]",1055917933902043,Test1,databricks-e2demofieldengwest,dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_1/,FAILED


#### Build Manifest metadata

In [0]:

to_prepend_1 = [StructField("src_manifest_path", StringType(), True), StructField("dest_manifest_path", StringType(), True)] 

s3_manifest_schema = StructType(df_s3_recon.schema.fields + to_prepend_1)

def build_s3_manifast_path_for_recon(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
   for pdf in iterator:
     #build manifest apth for both - src and destination
     src_inventory_s3_location = pdf["src_inventory_s3_location"][0]
     src_bucket_name = pdf["src_bucket_name"][0]
     src_prefix = pdf["src_prefix"][0]
     src_inventory_name = pdf["src_inventory_name"][0]
     dest_inventory_s3_location = pdf["dest_inventory_s3_location"][0]
     dest_bucket_name = pdf["dest_bucket_name"][0]
     dest_prefix = pdf["dest_prefix"][0]
     dest_inventory_name = pdf["dest_inventory_name"][0]
     pdf["src_manifest_path"] = f"{src_inventory_s3_location}/{src_bucket_name}/{src_inventory_name}"
     pdf["dest_manifest_path"] = f"{dest_inventory_s3_location}/{dest_bucket_name}/{dest_inventory_name}"
   yield pdf

df_s3_recon_with_manifest_path = df_s3_recon.mapInPandas(build_s3_manifast_path_for_recon, s3_manifest_schema)
df_s3_recon_with_manifest_path.display() if trace else None

dest_bucket_name,dest_inventory_name,dest_inventory_s3_location,dest_prefix,enabled,last_recon_timestamp,recon_status,runId,s3_batch_copy_id,src_bucket_name,src_inventory_name,src_inventory_s3_location,src_prefix,status,src_manifest_path,dest_manifest_path
databricks-e2demofieldengwest,dom_rodrigues_inventory_dest_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_2/,true,2025-11-10T16:21:14.828907Z,"[""{""status"":""EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"",""count"":44}""]",1055917933902043,Test1,databricks-e2demofieldengwest,dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_1/,FAILED,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test


In [0]:

from pyspark.sql.functions import col
src_s3_recon_df = df_s3_recon_with_manifest_path.select(col("src_bucket_name").alias("bucket_name"), col("src_prefix").alias("prefix"),col("src_inventory_name").alias("inventory_name"), col("src_inventory_s3_location").alias("inventory_s3_location"), col("src_manifest_path").alias("manifest_path"))

dest_s3_recon_df = df_s3_recon_with_manifest_path.select(col("dest_bucket_name").alias("bucket_name"),col("dest_prefix").alias("prefix"),col("dest_inventory_name").alias("inventory_name"), col("dest_inventory_s3_location").alias("inventory_s3_location"), col("dest_manifest_path").alias("manifest_path")) 

src_s3_recon_df.limit(10).display() if trace else None
dest_s3_recon_df.limit(10).display() if trace else None

bucket_name,prefix,inventory_name,inventory_s3_location,manifest_path
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test


bucket_name,prefix,inventory_name,inventory_s3_location,manifest_path
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,dom_rodrigues_inventory_dest_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test


### Find latest inventory folder for src and dest and save into inventory list table

In [0]:

inventory_reports_schema = StructType([
  StructField("file", StringType(), True),
  StructField("name", StringType(), True),
  StructField("size", DoubleType(), True),
  StructField("modificationTime", TimestampType(), True)])

import re
regex = r"^(.*)/(.*)/"

# path1= "s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/dom-rodrigues-us-east1-dbx-s3-1/dom_dbx_s3_1_notebooks/2025-10-18T01-00Z/"
#print(re.search(regex, path1).group(1))


def get_inventory_list(s3_recon_df):

  def get_list(manifest_path):
    inventory_reports_list = dbutils.fs.ls( manifest_path)

    inventory_reports = [{"file":re.search(regex, file.path).group(1), "name": file.name, "size": file.size, "modificationTime": int(file.modificationTime/1000)} for file in inventory_reports_list if file.name not in ["hive/", "data/"]]
    json_dump =  json.dumps(inventory_reports) #for arrow
    return json_dump


  inventory_df = pd.DataFrame(columns=s3_recon_df.columns)

  for index, row in s3_recon_df.toPandas().iterrows():
    list_of_files = get_list(row["manifest_path"])
    row["list_of_files"] = list_of_files 
    inventory_df = pd.concat([inventory_df, pd.DataFrame([row])], ignore_index=True)

  inventory_df_spark = spark.createDataFrame(inventory_df)
  return inventory_df_spark 

In [0]:
def recon_inventory_files_with_selected_column(s3_recon_df):

  recon_df_with_list = get_inventory_list(s3_recon_df) 
  recon_df_with_list_exploded =  recon_df_with_list.withColumn("inventory_reports", explode(from_json("list_of_files", ArrayType(StringType()))))

  recon_df_with_selected_columns =  recon_df_with_list_exploded\
    .withColumn("inventory_reports_details", from_json(col("inventory_reports"), inventory_reports_schema))\
    .select("bucket_name", "prefix", "inventory_s3_location",  "inventory_reports_details.file", "inventory_reports_details.size", "inventory_reports_details.name", "inventory_reports_details.modificationTime")\
    .withColumnRenamed("file", "path")

  return recon_df_with_selected_columns

In [0]:

# Pick latest after stripping the date from path s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/dom-rodrigues-us-east1-dbx-s3-1/dom_dbx_s3_1_notebooks/2025-10-18T01-00Z/

src_recon_df_with_selected_columns = recon_inventory_files_with_selected_column(src_s3_recon_df)
dest_recon_df_with_selected_columns = recon_inventory_files_with_selected_column(dest_s3_recon_df)

#.withColumn("stripped_path", F.regexp_extract(F.col("path"), "^(.*)/(.*)/", 1))\

recon_df_with_selected_columns = src_recon_df_with_selected_columns.unionByName(dest_recon_df_with_selected_columns)\
  .withColumn("Rank",  row_number().over(Window.partitionBy("path").orderBy(F.desc("name"))))\
  .filter(col("Rank") == 1).drop("Rank")

recon_df_with_selected_columns.display() if trace else None


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z


In [0]:
if createMetadataTables:
  #spark.sql(f"drop table if exists {catalog}.{schema}.ucd_s3_inventory_list")
  recon_df_with_selected_columns.withColumn("processed", lit("false")).withColumn("runId", lit("")).limit(0).write.option("mergeSchema", "true").mode("overwrite").saveAsTable(f"{catalog}.{schema}.ucd_s3_inventory_list")
spark.sql(f"select *  from {catalog}.{schema}.ucd_s3_inventory_list").limit(2).display() if trace else None

bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-07T01-00Z/,2025-11-07T21:05:25Z,true,83004138433296
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-07T01-00Z/,2025-11-07T21:05:23Z,true,83004138433296


In [0]:
recon_df_with_selected_columns.display() if trace else None
existing_inventory_list_df = spark.sql(f"select *  from {catalog}.{schema}.ucd_s3_inventory_list")
existing_inventory_list_df.filter(col("processed") == "false").display() if trace else None


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:34:26Z,false,75826bee-85ef-4467-933e-3fbfa12ee390
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:34:26Z,false,75826bee-85ef-4467-933e-3fbfa12ee390


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, concat, col

delta_tgt = DeltaTable.forName(spark, f"{catalog}.{schema}.ucd_s3_inventory_list")
delta_tgt.toDF().filter(col("processed") == "false").display()

delta_tgt.alias("tgt").merge(
   recon_df_with_selected_columns.withColumn("processed", lit("false")).alias("src")\
       .withColumn("runId", F.lit(runId)),
   "tgt.bucket_name = src.bucket_name and tgt.prefix = src.prefix and tgt.path == src.path"
) .whenMatchedUpdate(condition="tgt.processed =='false'", set={"processed": lit("ignored"), "runId": F.concat(col("tgt.runId"), F.lit(":"),lit(runId))}) \
.execute()

delta_tgt.toDF().filter(col("processed") == "false").display()
delta_tgt.alias("tgt").merge(
   recon_df_with_selected_columns.withColumn("processed", lit("false")).alias("src")\
       .withColumn("runId", F.lit(runId)),
   "tgt.bucket_name = src.bucket_name and tgt.prefix = src.prefix and tgt.path == src.path and tgt.processed == false"
).whenNotMatchedInsertAll() \
.execute()

delta_tgt.toDF().filter(col("processed") == "false").display()



bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:34:26Z,false,75826bee-85ef-4467-933e-3fbfa12ee390
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:34:26Z,false,75826bee-85ef-4467-933e-3fbfa12ee390


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


In [0]:
s3_inventory_manifest_df = delta_tgt.toDF()
s3_inventory_manifest_df.filter(col("processed") == "false").limit(2).display() if trace else None

bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


In [0]:
s3_inventory_files_to_process = s3_inventory_manifest_df.filter(col("processed") == "false")
if (s3_inventory_files_to_process.count() == 0):
  message = f"No new inventory files to process  for s3_batch_copy_id {s3_batch_copy_id} using  runId {runId}"
  raise Exception(json.dumps( {"status": "WARNING", "message": message}))
else:
  s3_inventory_files_to_process.display() if trace else None  

bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


### For the latest inventory, read gz file and build inventory list_details table

In [0]:
print(f"Starting recon process for {s3_batch_copy_id} using runId {runId}")

Starting recon process for Test1 using runId d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

def read_manifest_json(manifest_path_json):
  df = spark.read.option("multiline", "true").json(manifest_path_json)
  file_format = df.select("fileFormat").first()["fileFormat"]
  file_schema = df.select("fileSchema").first()["fileSchema"]

  # Explode the files array to get individual file rows
  files_df = df.select(explode("files").alias("file"))
  files_df = files_df.select(
    col("file.key").alias("key"),
    col("file.size").alias("size"),
    col("file.MD5checksum").alias("md5")
  )

  # Collect file paths as a Python list
  file_keys = [row["key"] for row in files_df.collect()]

  print("File Format:", file_format)
  print("File Keys:", file_keys)
  return {"file_key": file_keys[0], "file_format": file_format, "file_schema": file_schema}


#convert to pandas dataframe to use spark context
s3_inventory_manifest_df_result = pd.DataFrame(columns=s3_inventory_files_to_process.columns)

for index, row in s3_inventory_files_to_process.toPandas().iterrows():
    try: 
      row["file_info"] = read_manifest_json(row["path"] + "/" +  row["name"] + "/manifest.json")
      s3_inventory_manifest_df_result = pd.concat([s3_inventory_manifest_df_result, pd.DataFrame([row])], ignore_index=True)
    except Exception as e:
      print(e)  

s3_inventory_manifest_df_result_spark = spark.createDataFrame(s3_inventory_manifest_df_result)
s3_inventory_manifest_df_result_spark.display() if debug else None      

File Format: CSV
File Keys: ['dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test/data/3e59a1d6-8c77-4acb-86db-155af6767409.csv.gz']
File Format: CSV
File Keys: ['dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test/data/611550a3-b035-4a51-9abb-b652307e1d89.csv.gz']


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId,file_info
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,"List(CSV, dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test/data/3e59a1d6-8c77-4acb-86db-155af6767409.csv.gz, Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm)"
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,"List(CSV, dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test/data/611550a3-b035-4a51-9abb-b652307e1d89.csv.gz, Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm)"


In [0]:
# file_schema = "Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm"
# customSchema =  StructType([StructField(field.strip(), StringType(), True) for field in file_schema.split(",")])

#   # Path to the gzipped CSV file
# gz_path = "s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test/data/e63fff2f-1b6f-4eaa-8014-5c561205c074.csv.gz"
# dbutils.fs.ls(gz_path)
# df = spark.read.option("header", True).schema(customSchema).csv(gz_path)
# df.printSchema()
# df.display()

In [0]:

def read_gz_csv(row):
  inventory_s3_location = row["inventory_s3_location"] #s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY
  bucket_name = row["bucket_name"]
  prefix = row["prefix"]
              
  gz_csv_path = row["file_info"]["file_key"] #"INVENTORY/dom-rodrigues-us-east1-dbx-s3-1/dom_dbx_s3_1_notebooks/data/1fabdf54-8a55-4e97-af78-69a147d2fc71.csv.gz"
  file_format = row["file_info"]["file_format"]
  file_schema = row["file_info"]["file_schema"]
  customSchema =  StructType([StructField(field.strip(), StringType(), True) for field in file_schema.split(",")])

  # Path to the gzipped CSV file
  gz_path = f's3://{bucket_name}/{gz_csv_path}' #for DBX
  #gz_path = f'{inventory_s3_location}/{gz_csv_path}' #for JPMC
  print(gz_path) if debug else None

  if file_format == "CSV":
      df = spark.read.option("header", True).schema(customSchema).csv(gz_path)
  elif file_format == "Parquet":
      df = spark.read.parquet(*gz_path)
  elif file_format == "ORC":
      df = spark.read.orc(*gz_path)

  # Read gzipped CSV directly into DataFrame
  if df.isEmpty() == False:

    #Update prior records as ignored for this path
    path_ = row["path"]
    name_ = row["name"]
    spark.sql(f"UPDATE {catalog}.{schema}.ucd_s3_inventory_details set processed= 'ignore' \
     where bucket = '{bucket_name}' and inventory_s3_location = '{inventory_s3_location}' and path = '{path_}' and processed = 'false'")    
    
    if "IsLatest" in df.columns:
      df = df.filter((col('IsLatest') == True) | col('IsLatest').isNull() | (col('IsLatest') == ''))

    df_filtered_for_given_prefix = df.filter(col("Key").startswith(prefix))

    df_filtered_for_given_prefix.limit(2).display() if trace else None

    df_filtered_for_given_prefix.withColumn("inventory_s3_location", lit(row["inventory_s3_location"]))\
      .withColumn("path", lit(row["path"]))\
      .withColumn("name", lit(row["name"]))\
      .withColumn("prefix", lit(row["prefix"]))\
      .withColumn("processed", lit("false"))\
      .withColumn("runId", lit(runId))\
      .write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(f"{catalog}.{schema}.ucd_s3_inventory_details")
    return True
  else:
    return False



In [0]:

# file="s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/../INVENTORY/dom-rodrigues-us-east1-dbx-s3-1/dom_dbx_s3_1_notebooks/data/1fabdf54-8a55-4e97-af78-69a147d2fc71.csv.gz"
# df = spark.read.option("header", True).csv(file)
# df.count() if debug else None

In [0]:
gz_csv_df_result = pd.DataFrame(columns=s3_inventory_manifest_df_result_spark.columns)

#convert into pandas to use dbutils
for index, row in s3_inventory_manifest_df_result_spark.toPandas().iterrows():
    try: 
      row["result"] = read_gz_csv(row)
      gz_csv_df_result = pd.concat([gz_csv_df_result, pd.DataFrame([row])], ignore_index=True)
    except Exception as e:
      print(e)  

gz_csv_df_result.display() if debug else None  

s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test/data/3e59a1d6-8c77-4acb-86db-155af6767409.csv.gz


Bucket,Key,Size,LastModifiedDate,ETag,StorageClass,IsMultipartUploaded,ReplicationStatus,IntelligentTieringAccessTier,BucketKeyStatus,ChecksumAlgorithm
databricks-e2demofieldengwest,dom_rodrigues/folder_1/XGBoostTrainer_2025-07-16_21-43-17/,0,2025-07-16T21:43:19.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME
databricks-e2demofieldengwest,dom_rodrigues/folder_1/XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,2025-07-16T21:43:19.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME


s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test/data/611550a3-b035-4a51-9abb-b652307e1d89.csv.gz


Bucket,Key,Size,LastModifiedDate,ETag,StorageClass,IsMultipartUploaded,ReplicationStatus,IntelligentTieringAccessTier,BucketKeyStatus,ChecksumAlgorithm
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/,0,2025-11-03T20:55:02.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,2025-11-05T19:40:25.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME


/root/.ipykernel/3015/command-905161963805959-1121100956:7: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  gz_csv_df_result = pd.concat([gz_csv_df_result, pd.DataFrame([row])], ignore_index=True)


bucket_name,prefix,inventory_s3_location,path,size,name,modificationTime,processed,runId,file_info,result
databricks-e2demofieldengwest,dom_rodrigues/folder_1/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:15Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,"List(CSV, dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_test/data/3e59a1d6-8c77-4acb-86db-155af6767409.csv.gz, Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm)",true
databricks-e2demofieldengwest,dom_rodrigues/folder_2/,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,0.0,2025-11-10T01-00Z/,2025-11-11T16:36:16Z,false,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,"List(CSV, dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test/data/611550a3-b035-4a51-9abb-b652307e1d89.csv.gz, Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm)",true


In [0]:
spark.sql(f"select * from {catalog}.{schema}.ucd_s3_inventory_details").limit(10).display()
#Bucket, Key, Size, LastModifiedDate, ETag, StorageClass, IsMultipartUploaded, ReplicationStatus, IntelligentTieringAccessTier, BucketKeyStatus, ChecksumAlgorithm


Bucket,Key,Size,LastModifiedDate,ETag,StorageClass,IsMultipartUploaded,ReplicationStatus,IntelligentTieringAccessTier,BucketKeyStatus,ChecksumAlgorithm,inventory_s3_location,path,name,processed,runId,prefix,IsLatest
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/,0,2025-11-03T20:55:02.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,2025-11-05T19:40:25.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,2025-11-03T22:41:14.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,2025-11-05T19:40:58.000Z,d41d8cd98f00b204e9800998ecf8427e,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/basic-variant-state-2025-07-16_21-43-21.json,6905,2025-11-05T19:40:25.000Z,893385a5aa2cbabf348c171b4d74989c,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/experiment_state-2025-07-16_21-43-21.json,5538,2025-11-05T19:40:25.000Z,facb8e70f8d4f70f01adbbcb897b48d7,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/trainer.pkl,2200,2025-11-05T19:40:25.000Z,66edcb94f0a2af6190c965b06635b2ec,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY/databricks-e2demofieldengwest/dom_rodrigues_inventory_dest_test,2025-11-09T01-00Z/,ignore,6ddc778b-4d44-47d4-9f95-d841f60e3a1e,dom_rodrigues/folder_2/,null
databricks-e2demofieldengwest,dom_rodrigues/folder_2/XGBoostTrainer_2025-07-16_21-43-17/tuner.pkl,1242,2025-11-05T19:40:25.000Z,7b6ffec15856aa77fd72914d5b89467b,STANDARD,false,null,null,DISABLED,CRC64NVME,s3://databricks-e2demofieldengwest

#### Compare inventory details using batch copy id

### Create recon table using source and destination prefix

In [0]:
src_df_s3_recon = df_s3_recon.select(col("src_bucket_name").alias("bucket_name"), col("src_inventory_name").alias("inventory_name"), col("src_inventory_s3_location").alias("inventory_s3_location"), col("src_prefix").alias("prefix"), col("s3_batch_copy_id"), lit(runId).alias("runId"))
                                     
src_df_s3_recon.display() if trace else None
dest_df_s3_recon = df_s3_recon.select(col("dest_bucket_name").alias("bucket_name"), col("dest_inventory_name").alias("inventory_name"), col("dest_inventory_s3_location").alias("inventory_s3_location"), col("dest_prefix").alias("prefix"), col("s3_batch_copy_id"), lit(runId).alias("runId"))


bucket_name,inventory_name,inventory_s3_location,prefix,s3_batch_copy_id,runId
databricks-e2demofieldengwest,dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


In [0]:
ucd_s3_inventory_details_df_all = spark.sql(f"select * from {catalog}.{schema}.ucd_s3_inventory_details where processed = 'false'")
ucd_s3_inventory_details_df_src =  ucd_s3_inventory_details_df_all.alias("all").join(src_df_s3_recon.alias("src"), (col("all.Bucket") == col("src.bucket_name")) & (col("all.prefix") == col("src.prefix")) & (col("all.inventory_s3_location") == col("src.inventory_s3_location"))).select(col("all.Bucket"), F.regexp_replace(col("all.Key"), F.concat(F.lit("^"), col("all.prefix")), "").alias("Key"), col("Size"),  col("ETag"), col('all.prefix'), col("src.s3_batch_copy_id"), col("all.runId"))

ucd_s3_inventory_details_df_src.limit(5).display() if trace else None

ucd_s3_inventory_details_df_dest =  ucd_s3_inventory_details_df_all.alias("all").join(dest_df_s3_recon.alias("dest"), (col("all.Bucket") == col("dest.bucket_name")) & (col("all.prefix") == col("dest.prefix")) & (col("all.inventory_s3_location") == col("dest.inventory_s3_location"))).select(col("all.Bucket"), F.regexp_replace(col("all.Key"), F.concat(F.lit("^"), col("all.prefix")), "").alias("Key"), col("Size"),  col("ETag"), col('all.prefix'), col("dest.s3_batch_copy_id"), col("all.runId"))

ucd_s3_inventory_details_df_dest.limit(5).display() if trace else None


Bucket,Key,Size,ETag,prefix,s3_batch_copy_id,runId
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/basic-variant-state-2025-07-16_21-43-21.json,6905,893385a5aa2cbabf348c171b4d74989c,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/experiment_state-2025-07-16_21-43-21.json,5538,facb8e70f8d4f70f01adbbcb897b48d7,dom_rodrigues/folder_1/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


Bucket,Key,Size,ETag,prefix,s3_batch_copy_id,runId
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_2/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_2/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_2/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_2/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd
databricks-e2demofieldengwest,XGBoostTrainer_2025-07-16_21-43-17/basic-variant-state-2025-07-16_21-43-21.json,6905,893385a5aa2cbabf348c171b4d74989c,dom_rodrigues/folder_2/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd


In [0]:
match_columns = [  'Key','s3_batch_copy_id', 'runId']

existsInSrcAndNotInDestination = ucd_s3_inventory_details_df_src.join(ucd_s3_inventory_details_df_dest, on=match_columns, how='left_anti').withColumn("status", lit("EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"))
existsInSrcAndNotInDestination.limit(10).display() if trace else None

detailed_match_all_columns = [  'Key', "Size", "Etag",'s3_batch_copy_id', 'runId']
existsInSrcAndDestinationAndSame = ucd_s3_inventory_details_df_src.alias("src").join(ucd_s3_inventory_details_df_dest.alias("dest"), on=detailed_match_all_columns).select("src.*").withColumn("status", lit("EXISTS_IN_SRC_AND_DEST_SAME"))
existsInSrcAndDestinationAndSame.limit(10).display() if trace else None

detailed_match_columns = [ 'Key', 's3_batch_copy_id', 'runId']
existsInSrcAndDestinationAndButNotSame = ucd_s3_inventory_details_df_src.alias("src").join(ucd_s3_inventory_details_df_dest.alias("dest"), on=detailed_match_columns, how='inner').filter((col("src.size") != col("dest.size")) | (col("src.ETag") != col("dest.ETag"))).select("src.*").withColumn("status", lit("EXISTS_IN_SRC_AND_DEST_BUT_NOT_SAME"))
existsInSrcAndDestinationAndButNotSame.limit(10).display() if trace else None


Key,s3_batch_copy_id,runId,Bucket,Size,ETag,prefix,status
XGBoostTrainer_2025-07-16_21-43-18/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-43-19/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-43-42/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-44-32/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-45-32/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-46-02/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-16_21-48-54/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-17_21-11-12/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-17_21-11-17/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST
XGBoostTrainer_2025-07-17_21-11-18/,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,0,d41d8cd98f00b204e9800998ecf8427e,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST


Key,Size,ETag,s3_batch_copy_id,runId,Bucket,prefix,status
XGBoostTrainer_2025-07-16_21-43-17/,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/.validate_storage_marker,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/XGBoostTrainer_e32ff_00000_0_2025-07-16_21-43-21/,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/basic-variant-state-2025-07-16_21-43-21.json,6905,893385a5aa2cbabf348c171b4d74989c,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/experiment_state-2025-07-16_21-43-21.json,5538,facb8e70f8d4f70f01adbbcb897b48d7,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/trainer.pkl,2200,66edcb94f0a2af6190c965b06635b2ec,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-17/tuner.pkl,1242,7b6ffec15856aa77fd72914d5b89467b,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-18/.validate_storage_marker,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-18/XGBoostTrainer_e5f11_00000_0_2025-07-16_21-43-28/,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME
XGBoostTrainer_2025-07-16_21-43-18/XGBoostTrainer_e5f18_00000_0_2025-07-16_21-43-28/,0,d41d8cd98f00b204e9800998ecf8427e,Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,EXISTS_IN_SRC_AND_DEST_SAME


Key,s3_batch_copy_id,runId,Bucket,Size,ETag,prefix,status


In [0]:
detailed_match_columns = [ 'Key', 's3_batch_copy_id', 'runId']

existsInSrcAndDestinationButNotSame = ucd_s3_inventory_details_df_src.alias("src").join(ucd_s3_inventory_details_df_dest.alias("dest"), on=detailed_match_columns, how='inner').filter((col("src.size") != col("dest.size")) | (col("src.ETag") != col("dest.ETag"))).select("src.*", "dest.*").withColumn("status", lit("EXISTS_IN_SRC_AND_DEST_BUT_NOT_SAME"))
existsInSrcAndDestinationButNotSame.display()


Key,s3_batch_copy_id,runId,Bucket,Size,ETag,prefix,Key,s3_batch_copy_id,runId,Bucket,Size,ETag,prefix,status


In [0]:
if createMetadataTables:
  spark.sql(f"drop table if exists {catalog}.{schema}.ucd_s3_inventory_recon")
  existsInSrcAndNotInDestination.withColumnRenamed("bucket", "src_bucket").withColumnRenamed("prefix", "src_prefix").withColumn("time_stamp", current_timestamp()).withColumn("dest_bucket", F.lit("")).withColumn("dest_prefix", F.lit("")).limit(0).write.option("mergeSchema", "true").mode("overwrite").saveAsTable(f"{catalog}.{schema}.ucd_s3_inventory_recon")
spark.sql(f"select *  from {catalog}.{schema}.ucd_s3_inventory_recon")

DataFrame[Key: string, s3_batch_copy_id: string, runId: string, src_bucket: string, Size: string, ETag: string, src_prefix: string, status: string, time_stamp: timestamp, dest_bucket: string, dest_prefix: string, dest_bucket_name: string]

In [0]:

current_datetime = datetime.now()
recon_failed: bool = False

if (existsInSrcAndNotInDestination.isEmpty() == False):
    recon_failed = True
    existsInSrcAndNotInDestination.alias("inventory").join(df_s3_recon.alias("recon"), (col("inventory.Bucket") == col("recon.src_bucket_name")) & (col("inventory.s3_batch_copy_id") == col("recon.s3_batch_copy_id"))).select(col("inventory.*"), col("recon.dest_bucket_name"), col("recon.dest_prefix")).withColumn("time_stamp", lit(current_datetime)).withColumnRenamed("Bucket", "src_bucket").withColumnRenamed("prefix", "src_prefix")\
    .write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(f"{catalog}.{schema}.ucd_s3_inventory_recon")

if (existsInSrcAndDestinationAndButNotSame.isEmpty() == False):
    recon_failed = True
    existsInSrcAndDestinationAndButNotSame.alias("inventory").join(df_s3_recon.alias("recon"), (col("inventory.Bucket") == col("recon.src_bucket_name")) & (col("inventory.s3_batch_copy_id") == col("recon.s3_batch_copy_id"))).select(col("inventory.*"), col("recon.dest_bucket_name"), col("recon.dest_prefix")).withColumn("time_stamp", lit(current_datetime)).withColumnRenamed("Bucket", "src_bucket").withColumnRenamed("prefix", "src_prefix")\
    .write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(f"{catalog}.{schema}.ucd_s3_inventory_recon")




In [0]:
recon_df = spark.sql(f"select  s3_batch_copy_id, runId, src_bucket,  src_prefix, dest_bucket, dest_prefix, status, time_stamp, count(*) as count from {catalog}.{schema}.ucd_s3_inventory_recon where s3_batch_copy_id = '{s3_batch_copy_id}' and runId = '{runId}' and time_stamp = '{current_datetime}' group by s3_batch_copy_id, runId, src_bucket,  src_prefix, dest_bucket, dest_prefix, status, time_stamp").withColumn("status_json", F.to_json(F.struct("status", "count")))

recon_result_df = recon_df.groupBy("s3_batch_copy_id",  "runId", "src_bucket",
                    "src_prefix", "dest_bucket", "dest_prefix", "time_stamp").agg(
          F.collect_list("status_json").alias("status_json") 
      ) 

recon_result_df.display() if trace else None

recon_status = {"status": "SUCCESS"} if recon_result_df.select("status_json").first() == None  else recon_result_df.select("status_json").first()[0]
status = 'FAILED' if recon_failed else 'SUCCESS'

print(f"Recon status for s3_batch_copy_id {s3_batch_copy_id} for runId {runId} is  {status}, details: {recon_status}")

s3_batch_copy_id,runId,src_bucket,src_prefix,dest_bucket,dest_prefix,time_stamp,status_json
Test1,d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,null,dom_rodrigues/folder_2/,2025-11-11T16:36:40.919822Z,"List({""status"":""EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"",""count"":44})"


Recon status for s3_batch_copy_id Test1 for runId d31c2804-74c8-40ea-9dbb-9f4cf606bbdd is  FAILED, details: ['{"status":"EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST","count":44}']


In [0]:
spark.sql(f"select * from {catalog}.{schema}.ucd_src_dest_mapping").display()

dest_bucket_name,dest_inventory_name,dest_inventory_s3_location,dest_prefix,enabled,last_recon_timestamp,recon_status,runId,s3_batch_copy_id,src_bucket_name,src_inventory_name,src_inventory_s3_location,src_prefix,status
databricks-e2demofieldengwest,dom_rodrigues_inventory_dest_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_2/,true,2025-11-10T16:21:14.828907Z,"[""{""status"":""EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"",""count"":44}""]",1055917933902043,Test1,databricks-e2demofieldengwest,dom_rodrigues_inventory_test,s3://databricks-e2demofieldengwest/dom_rodrigues/INVENTORY,dom_rodrigues/folder_1/,FAILED


In [0]:
spark.sql(f"""UPDATE {catalog}.{schema}.ucd_src_dest_mapping set last_recon_timestamp = '{current_datetime}', runId = '{runId}',status = '{status}', recon_status = '{json.dumps(recon_status)}' where s3_batch_copy_id = '{s3_batch_copy_id}' and enabled=True""")

DataFrame[num_affected_rows: bigint]

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, concat, col

delta_tgt = DeltaTable.forName(spark, f"{catalog}.{schema}.ucd_s3_inventory_list")

delta_tgt.alias("tgt").merge(
    s3_inventory_files_to_process.alias("src"),
    "tgt.bucket_name = src.bucket_name and tgt.path == src.path and tgt.processed =='false'"
) .whenMatchedUpdate(set={"processed": lit("true"), "runId":  lit(runId)}) \
 .execute()


In [0]:
spark.sql(f"""select runId, s3_batch_copy_id, src_bucket_name, dest_bucket_name, src_prefix, dest_prefix, status, recon_status, last_recon_timestamp  from {catalog}.{schema}.ucd_src_dest_mapping  where s3_batch_copy_id = '{s3_batch_copy_id}' and enabled=True""").display()

runId,s3_batch_copy_id,src_bucket_name,dest_bucket_name,src_prefix,dest_prefix,status,recon_status,last_recon_timestamp
d31c2804-74c8-40ea-9dbb-9f4cf606bbdd,Test1,databricks-e2demofieldengwest,databricks-e2demofieldengwest,dom_rodrigues/folder_1/,dom_rodrigues/folder_2/,FAILED,"[""{""status"":""EXISTS_IN_SRC_AND_BUT_NOT_IN_DEST"",""count"":44}""]",2025-11-11T16:36:40.919822Z


In [0]:
dbutils.notebook.exit(json.dumps( {"status": f'{status}', "message": f'{recon_status}', 'runId': runId}))